# Graph Coloring

The **"Graph Coloring"** quantum kata is a series of exercises designed
to teach you the basics of using Grover search to solve constraint
satisfaction problems, using graph coloring problem as an example.

* [This Microsoft Learn module](https://docs.microsoft.com/learn/modules/solve-graph-coloring-problems-grovers-search/) walks you through solving graph coloring problems using Grover's search.
* You can read more about graph coloring problems [here](https://en.wikipedia.org/wiki/Graph_coloring).
* It is strongly recommended to complete the [Grover's Algorithm kata](./../GroversAlgorithm/GroversAlgorithm.ipynb) before proceeding to this one. You can also refer to its [README.md](./../GroversAlgorithm/README.md) for the list of resources on Grover's algorithm.
* [SolveSATWithGrover](./../SolveSATWithGrover/SolveSATWithGrover.ipynb) is another kata covering oracle implementation for solving constraint satisfaction problems.


Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with the `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl+Enter (⌘+Enter on macOS).

Within each section, tasks are given in approximate order of increasing difficulty; 
harder ones are marked with asterisks.

## Part I. Colors Representation and Manipulation

### Task 1.1. Initialize register to a color

**Inputs:** 

  1. An integer $C$ ($0 \leq C \leq 2^{N} - 1$).

  2. An array of $N$ qubits in the $|0...0\rangle$ state.

**Goal:** 

Prepare the array in the basis state which represents the binary notation of $C$. 
Use little-endian encoding (i.e., the least significant bit should be stored in the first qubit).

**Example:** For $N = 2$ and $C = 2$ the state should be $|01\rangle$.

In [ ]:
%kata T11_InitializeColor 

open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Logical;

operation InitializeColor (C : Int, register : Qubit[]) : Unit is Adj {
    //
    let N = Length(register);
    let bitString = IntAsBoolArray(C, N);
    //for i in 0 .. N - 1 {
    //    if (bitString[i]) {
    //        X(register[i]);
    //    }
    //}
    ApplyPauliFromBitString(PauliX, true, bitString, register);
}

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-1.1.-Initialize-register-to-a-color).*

### Task 1.2. Read color from a register

**Input:** An array of $N$ qubits which are guaranteed to be in one of the $2^{N}$ basis states.

**Output:** 

An $N$-bit integer that represents this basis state, in little-endian encoding. 
The operation should not change the state of the qubits.

**Example:** For $N = 2$ and the qubits in the state $|01\rangle$ return 2 (and keep the qubits in $|01\rangle$).

In [23]:
%kata T12_MeasureColor 

open Microsoft.Quantum.Measurement;

operation MeasureColor (register : Qubit[]) : Int {
    // 
    return ResultArrayAsInt(MultiM(register));
}

Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-1.2.-Read-color-from-a-register).*

### Task 1.3. Read coloring from a register

**Inputs:** 

  1. The number of elements in the coloring $K$.

  2. An array of $K * N$ qubits which are guaranteed to be in one of the $2^{KN}$ basis states.

**Output:** 

An array of $K$ $N$-bit integers that represent this basis state. 
$i$-th integer of the array is stored in qubits with indices $i * N$, $i * N + 1$, ..., $i * N + N - 1$ in little-endian format. 
The operation should not change the state of the qubits.

**Example:** 
For $N = 2$, $K = 2$ and the qubits in the state $|0110\rangle$ return `[2, 1]`.

In [24]:
%kata T13_MeasureColoring 

operation MeasureColoring (K : Int, register : Qubit[]) : Int[] {
    // Functional solution is best
    let N = Length(register) / K;
    //mutable result = [0, size = K];
    //for i in 0 .. K - 1 {
    //    set result w/= i <- MeasureColor(register[i * N .. i * N + N - 1]);
    //}
    //return result;

    let colorPartitions = Chunks(N, register);
    // ForEach is for operations and Mapped is for functions
    let coloring = ForEach<Qubit[], Int>(MeasureColor(_), colorPartitions);
    return coloring;
}

Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-1.3.-Read-coloring-from-a-register).*

### Task 1.4. 2-bit color equality oracle

**Inputs:** 

  1. An array of 2 qubits in an arbitrary state $|c_{0}\rangle$ representing the first color.

  2. An array of 2 qubits in an arbitrary state $|c_{1}\rangle$ representing the second color.

  3. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|c_{0}\rangle|c_{1}\rangle|y\rangle$ into state $|c_{0}\rangle|c_{1}\rangle|y \oplus f(c_{0},c_{1})\rangle$ ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if $c_{0}$ and $c_{1}$ are in the same state, and 0 otherwise. 
Leave the query register in the same state it started in.

In this task you are allowed to allocate extra qubits.

In [ ]:
%kata T14_ColorEqualityOracle_2bit 

operation ColorEqualityOracle_2bit (c0 : Qubit[], c1 : Qubit[], target : Qubit) : Unit is Adj+Ctl {
    //
    let N = Length(c0);
    use q = Qubit[N];
    within {
        for i in 0 .. N - 1 {
            ControlledOnInt(0, X)(c0[i..i] + c1[i..i], q[i]);
            Controlled X(c0[i..i] + c1[i..i], q[i]);
        }
    }
    apply { Controlled X(q, target); }
}

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-1.4.-2-bit-color-equality-oracle).*

### Task 1.5. N-bit color equality oracle (no extra qubits)

This task is the same as task 1.4, but in this task you are NOT allowed to allocate extra qubits.

In [12]:
%kata T15_ColorEqualityOracle_Nbit 

operation ColorEqualityOracle_Nbit (c0 : Qubit[], c1 : Qubit[], target : Qubit) : Unit is Adj+Ctl {
    // 2N time complexity, but not tail recursive
    //if (IsEmpty(c0)) { X(target); }
    //else {
    //    ControlledOnInt(0, ColorEqualityOracle_Nbit)(c0[0..0] + c1[0..0], (Rest(c0), Rest(c1), target));
    //    Controlled ColorEqualityOracle_Nbit(c0[0..0] + c1[0..0], (Rest(c0), Rest(c1), target));
    //}
    // Beautiful
    within {
        for i in 0 .. Length(c0) - 1 {
            CNOT(c0[i], c1[i]);
        }
    } apply { ControlledOnInt(0, X)(c1, target); }
}

Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-1.5.-N-bit-color-equality-oracle-(no-extra-qubits)).*

## Part II. Vertex coloring problem
The vertex graph coloring is a coloring of graph vertices which 
labels each vertex with one of the given colors so that 
no two vertices of the same color are connected by an edge.

### Task 2.1. Classical verification of vertex coloring

**Inputs:** 

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. An array of $E$ tuples of integers, representing the edges of the graph ($E \leq 12$).  
Each tuple gives the indices of the start and the end vertices of the edge.  
The vertices are indexed $0$ through $V - 1$.

  3. An array of $V$ integers, representing the vertex coloring of the graph. 
$i$-th element of the array is the color of the vertex number $i$.

**Output:** 

True if the given vertex coloring is valid (i.e., no pair of vertices connected by an edge have the same color), and false otherwise.

**Example:** 

Graph 0 -- 1 -- 2 would have $V = 3$ and `edges = [(0, 1), (1, 2)]`.  
Some of the valid colorings for it would be `[0, 1, 0]` and `[-1, 5, 18]`.

In [ ]:
%kata T21_IsVertexColoringValid 

function IsVertexColoringValid (V : Int, edges: (Int, Int)[], colors: Int[]) : Bool {
    // Replace start/end in edges with the index values from colors
    for (start, end) in edges {
        if (colors[start] == colors[end]) {
            return false;
        }
    }
    return true;
}

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-2.1.-Classical-verification-of-vertex-coloring).*

### Task 2.2. Oracle for verifying vertex coloring

**Inputs:**

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. An array of $E$ tuples of integers, representing the edges of the graph (E $\leq$ 12).  
Each tuple gives the indices of the start and the end vertices of the edge.  
The vertices are indexed $0$ through $V - 1$.

  3. An array of $2V$ qubits `colorsRegister` that encodes the color assignments.

  4. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$  ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if the given vertex coloring is valid, and 0 otherwise. 
Leave the query register in the same state it started in.

Each color in `colorsRegister` is represented as a 2-bit integer in little-endian format. 
See task 1.3 for a more detailed description of color assignments.

In [ ]:
%kata T22_VertexColoringOracle 

operation VertexColoringOracle (
    V : Int, 
    edges : (Int, Int)[], 
    colorsRegister : Qubit[], 
    target : Qubit ) : Unit is Adj+Ctl {
    // 
    let E = Length(edges);
    use aux = Qubit[E];
    let colorPartitions = Chunks(2, colorsRegister);
    within {
        for i in 0 .. E - 1 {
            let (start, end) = edges[i];
            // Flip aux if any edge connects the same color
            // At the end, we flip only the 0 state
            ColorEqualityOracle_Nbit(colorPartitions[start], colorPartitions[end], aux[i]);
        }
    }
    apply { ControlledOnInt(0, X)(aux, target); }
}

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-2.2.-Oracle-for-verifying-vertex-coloring).*

### Task 2.3. Using Grover's search to find vertex coloring

**Inputs:** 

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. A marking oracle which implements vertex coloring verification, as implemented in task 2.2.

**Output:** 

A valid vertex coloring for the graph, in a format used in task 2.1.

In [1]:
operation PhaseOracle (markingOracle : ((Qubit[], Qubit) => Unit is Adj), register : Qubit[]) : Unit is Adj {
    // 
    use q = Qubit();
    within {
        X(q);
        H(q);
    } 
    apply { markingOracle(register, q); }
}

PhaseOracle

In [2]:
function OracleConverter (markingOracle : ((Qubit[], Qubit) => Unit is Adj)) : (Qubit[] => Unit is Adj) {
    // marking oracle on input with ancilla = phase oracle on input
    return PhaseOracle(markingOracle, _);
}

OracleConverter

In [3]:
operation HadamardTransform (register : Qubit[]) : Unit is Adj {
    //
    ApplyToEachCA(H, register);
}

HadamardTransform

In [4]:
operation Oracle_ArbitraryPattern (queryRegister : Qubit[], target : Qubit, pattern : Bool[]) : Unit is Adj + Ctl {
    //
    ControlledOnBitString(pattern, X)(queryRegister, target);
}

Oracle_ArbitraryPattern

In [6]:
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arrays;

operation ConditionalPhaseFlip (register : Qubit[]) : Unit is Adj {
    // Controlled marking oracle
    let ControlledMarkingOracle = Oracle_ArbitraryPattern(_, _, [false, size = Length(register)]);
    let ControlledPhaseOracle = OracleConverter(ControlledMarkingOracle);
    // Controlled phase kickback
    ControlledPhaseOracle(register);
    // Applying a global phase flip, even though it's unphysical(?)
    // It doesn't matter for Grover's search algorithm itself, since the global phase is not observable, 
    // but can have side effects when used as part of other algorithms.
    R(PauliI, 2.0 * PI(), Head(register));
}

ConditionalPhaseFlip

In [7]:
operation GroverIteration (register : Qubit[], oracle : (Qubit[] => Unit is Adj)) : Unit is Adj {
    //
    oracle(register);
    within { HadamardTransform(register); }
    apply { ConditionalPhaseFlip(register); }
}

GroverIteration

In [8]:
operation GroversSearch (register : Qubit[], oracle : ((Qubit[], Qubit) => Unit is Adj), iterations : Int) : Unit {
    //
    ApplyToEachCA(H, register);
    RepeatA(GroverIteration, iterations, (register, OracleConverter(oracle)));
}

GroversSearch

In [26]:
%kata T23_GroversAlgorithm 

open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;

operation GroversAlgorithm (V : Int, oracle : ((Qubit[], Qubit) => Unit is Adj)) : Int[] {
    //
    use (register, q) = (Qubit[2 * V], Qubit());
    mutable coloring = [0, size = V];
    mutable iterations = 1;

    repeat {
        GroversSearch(register, oracle, iterations);
        set coloring = MeasureColoring(V, register);
        oracle(register, q);
        let eigenstate = MResetZ(q);
        ResetAll(register);
    } 
    until (eigenstate == One or iterations > 10)
    fixup { set iterations *= 2; }

    return coloring;
}

Running on graph V = 3, edges = []
Got correct coloring [2,2,1]
Running on graph V = 4, edges = [(0, 1),(0, 2),(0, 3),(1, 2),(1, 3),(2, 3)]
Got correct coloring [3,2,0,1]
Running on graph V = 5, edges = [(4, 0),(2, 1),(3, 1),(3, 2)]
Got correct coloring [3,1,2,3,2]
Running on graph V = 5, edges = [(0, 1),(1, 2),(1, 3),(3, 2),(4, 2),(3, 4)]
Got correct coloring [0,3,0,1,3]
Running on graph V = 5, edges = [(0, 1),(0, 2),(0, 4),(1, 2),(1, 3),(2, 3),(2, 4),(3, 4)]
Got correct coloring [0,1,2,0,3]
Running on graph V = 6, edges = [(0, 1),(0, 2),(0, 4),(0, 5),(1, 2),(1, 3),(1, 5),(2, 3),(2, 4),(3, 4),(3, 5),(4, 5)]
Got correct coloring [0,1,3,0,2,3]


Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-2.3.-Using-Grover's-search-to-find-vertex-coloring).*

## Part III. Weak coloring problem
Weak graph coloring is a coloring of graph vertices which 
labels each vertex with one of the given colors so that 
each vertex is either isolated or is connected by an edge
to at least one neighbor of a different color.

### Task 3.1. Determine if an edge contains the vertex

**Inputs:**

   1. An edge denoted by a tuple of integers.
      Each tuple gives the indices of the start and the end vertices of the edge.
   2. An integer denoting the vertex of the graph.

**Output:** 

True if the edge contains the vertex provided, and false otherwise.

**Examples:** 
- edge $(0,1)$ contains vertex $0$, so return true;
- edge $(0,1)$ contains vertex $1$, so return true;
- edge $(2,3)$ does not contain vertex $1$, so return false.

In [2]:
%kata T31_DoesEdgeContainVertex

function DoesEdgeContainVertex (edge: (Int, Int), vertex : Int) : Bool {
    //
    return Fst(edge) == vertex or Snd(edge) == vertex;
}

Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-3.1.-Determine-if-an-edge-contains-the-vertex).*

### Task 3.2. Determine if a vertex is weakly colored
**Inputs:** 

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. An array of $E$ tuples of integers, representing the edges of the graph ($E \leq 12$).  
Each tuple gives the indices of the start and the end vertices of the edge.  
The vertices are indexed $0$ through $V - 1$.

  3. An array of $V$ integers, representing the vertex coloring of the graph. 
$i$-th element of the array is the color of the vertex number $i$.

  4. A vertex in the graph, indexed $0$ through $V - 1$.

**Output:**

True if the vertex is weakly colored (i.e., it is connected to at least one neighboring vertex of different color),
and false otherwise.

**Note:**

An isolated vertex (a vertex without neighbors) is considered to be weakly colored.

**Example:**

For vertex $0$, in a graph containing `edges = [(0, 1), (0, 2), (1, 2)]` and  `colors = [0, 1, 0]`, vertex $0$ is weakly colored, since it has color 0 and is connected to vertex 1 which has color 1.

In [8]:
%kata T32_IsVertexWeaklyColored

function IsVertexWeaklyColored (V : Int, edges: (Int, Int)[], colors: Int[], vertex : Int) : Bool {
    // Take into account isolation
    mutable isEmpty = true;

    for edge in edges {
        if (DoesEdgeContainVertex(edge, vertex)) {
            set isEmpty = false;
            if (colors[Fst(edge)] != colors[Snd(edge)]) {
                return true;
            }
        }
    }
    return isEmpty;
}

Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-3.2.-Determine-if-a-vertex-is-weakly-colored).*

### Task 3.3. Classical verification of weak coloring

**Inputs:** 

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. An array of $E$ tuples of integers, representing the edges of the graph ($E \leq 12$).  
Each tuple gives the indices of the start and the end vertices of the edge.  
The vertices are indexed $0$ through $V - 1$.

  3. An array of $V$ integers, representing the vertex coloring of the graph. 
$i$-th element of the array is the color of the vertex number $i$.

**Output:** 

True if the given weak coloring is valid (i.e., every vertex is isolated or is connected to at least one neighboring vertex of different color), and false otherwise.

**Example:** 

Consider a graph with $V = 3$ and `edges = [(0, 1), (0, 2), (1, 2)]`.  
Some of the valid colorings for it would be `[0, 1, 0]` and `[-1, 5, 18]`.

In [10]:
%kata T33_IsWeakColoringValid

open Microsoft.Quantum.Convert;

function IsWeakColoringValid (V : Int, edges: (Int, Int)[], colors: Int[]) : Bool {
    //
    //mutable weakColorings = 0;
    for i in 0 .. V - 1 {
        if (not IsVertexWeaklyColored(V, edges, colors, i)) {
            //set weakColorings += 1;
            return false;
        }
    }
    //return weakColorings == V;
    return true;
}

Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-3.3.-Classical-verification-of-weak-coloring).*

### Task 3.4. Oracle for verifying if a vertex is weakly colored

**Inputs:**

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. An array of $E$ tuples of integers, representing the edges of the graph (E $\leq$ 12).  
Each tuple gives the indices of the start and the end vertices of the edge.  
The vertices are indexed $0$ through $V - 1$.

  3. An array of $2V$ qubits `colorsRegister` that encodes the color assignments.

  4. A qubit in an arbitrary state $|y\rangle$ (target qubit).
  
  5. A vertex in the graph, indexed $0$ through $V - 1$.

**Goal:**

Transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$  ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if the given weak coloring is valid, and 0 otherwise. 
Leave the query register in the same state it started in.

Each color in `colorsRegister` is represented as a 2-bit integer in little-endian format. 
See $Task\ 1.3$ for a more detailed description of color assignments.

In [13]:
%kata T34_WeaklyColoredVertexOracle

open Microsoft.Quantum.Arrays;

operation WeaklyColoredVertexOracle(V : Int, edges: (Int, Int)[], colorsRegister : Qubit[], target : Qubit, vertex : Int) : Unit is Adj+Ctl {
    // If vertex is isolated, or contains at least 1 element of a different color
    // Filter out edges not connected to this vertex.
    let predicate = DoesEdgeContainVertex(_, vertex);
    let connectingEdges = Filtered(predicate, edges);
    let colors = Chunks(2, colorsRegister);
    let N = Length(connectingEdges);
    use aux = Qubit[N];
    within {
        // Mark all neighbors which are of the SAME color.
        for ((start, end), q) in Zipped(connectingEdges, aux) {
            ColorEqualityOracle_Nbit(colors[start], colors[end], q);
        }
    } 
    apply {
        // If any neighbor has a different color (i.e., at least one qubit is zero), flip target.
        // In other words, don't flip only for |1..1⟩.
        // (Remember that for N = 0, isolated vertex is ok, so target needs to be flipped as well.)
        X(target);
        if (N != 0) { 
            Controlled X(aux, target);
        }
    }
}

Testing V = 3, edges = []
Testing V = 3, edges = []
Testing V = 3, edges = []
Testing V = 4, edges = [(0, 1),(0, 2),(0, 3),(1, 2),(1, 3),(2, 3)]
Testing V = 4, edges = [(0, 1),(0, 2),(0, 3),(1, 2),(1, 3),(2, 3)]
Testing V = 4, edges = [(0, 1),(0, 2),(0, 3),(1, 2),(1, 3),(2, 3)]
Testing V = 4, edges = [(0, 1),(0, 2),(0, 3),(1, 2),(1, 3),(2, 3)]
Testing V = 5, edges = [(4, 0),(2, 1),(3, 1),(3, 2)]
Testing V = 5, edges = [(4, 0),(2, 1),(3, 1),(3, 2)]
Testing V = 5, edges = [(4, 0),(2, 1),(3, 1),(3, 2)]
Testing V = 5, edges = [(4, 0),(2, 1),(3, 1),(3, 2)]
Testing V = 5, edges = [(4, 0),(2, 1),(3, 1),(3, 2)]
Testing V = 5, edges = [(0, 1),(1, 2),(1, 3),(3, 2),(4, 2),(3, 4)]
Testing V = 5, edges = [(0, 1),(1, 2),(1, 3),(3, 2),(4, 2),(3, 4)]
Testing V = 5, edges = [(0, 1),(1, 2),(1, 3),(3, 2),(4, 2),(3, 4)]
Testing V = 5, edges = [(0, 1),(1, 2),(1, 3),(3, 2),(4, 2),(3, 4)]
Testing V = 5, edges = [(0, 1),(1, 2),(1, 3),(3, 2),(4, 2),(3, 4)]
Testing V = 5, edges = [(0, 1),(0, 2),(0, 4),(1, 2),(1

Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-3.4.-Oracle-for-verifying-if-a-vertex-is-weakly-colored).*

### Task 3.5. Oracle for verifying weak coloring
    
**Inputs:**

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. An array of $E$ tuples of integers, representing the edges of the graph ($E \leq 12$).  
Each tuple gives the indices of the start and the end vertices of the edge.  
The vertices are indexed $0$ through $V - 1$.

  3. An array of $2V$ qubits `colorsRegister` that encodes the color assignments.

  4. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$  ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if the given weak coloring is valid, and 0 otherwise. 
Leave the query register in the same state it started in.

Each color in `colorsRegister` is represented as a 2-bit integer in little-endian format. 
See $Task\ 1.3$ for a more detailed description of color assignments.

In [ ]:
%kata T35_WeakColoringOracle

operation WeakColoringOracle (
    V : Int,
    edges : (Int, Int)[],
    colorsRegister : Qubit[],
    target : Qubit
) : Unit is Adj+Ctl {
    // ...
}

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-3.5.-Oracle-for-verifying-weak-coloring).*

### Task 3.6. Using Grover's search to find weak coloring

**Inputs:** 

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. A marking oracle which implements weak coloring verification, as implemented in Task 3.5.

**Output:** 

A valid weak coloring for the graph, in a format used in Task 3.3.

In [ ]:
%kata T36_GroversAlgorithmForWeakColoring

operation GroversAlgorithmForWeakColoring (V : Int, oracle : ((Qubit[], Qubit) => Unit is Adj)) : Int[] {
    // ...
}

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-3.6.-Using-Grover's-search-to-find-weak-coloring).*

## Part IV. Triangle-free coloring problem

Triangle-free graph coloring is a coloring of graph edges which labels each edge with one of two colors so that no three edges of the same color form a triangle.

### Task 4.1. Convert the list of graph edges into an adjacency matrix

**Inputs:**

   1. The number of vertices in the graph $V$ ($V \leq 6$).
   
   2. An array of $E$ tuples of integers, representing the edges of the graph ($E \leq 12$). Each tuple gives the indices of the start and the end vertices of the edge. The vertices are indexed $0$ through $V - 1$.

**Output:**  
A 2D array of integers representing this graph as an adjacency matrix:  the element [i][j] should be $-1$ if the vertices i and j are not connected with an edge, or store the index of the edge if the vertices i and j are connected with an edge. Elements [i][i] should be $-1$ unless there is an edge connecting vertex i to itself.

**Example:**
Consider a graph with $V = 3$ and edges = `[(0, 1), (0, 2), (1, 2)]`.
The adjacency matrix for it would be:

$$
\begin{bmatrix}
-1 &  0 &  1 \\
 0 & -1 &  2 \\
 1 &  2 & -1
\end{bmatrix}
$$


In [ ]:
%kata T41_EdgesListAsAdjacencyMatrix

function EdgesListAsAdjacencyMatrix (V : Int, edges : (Int, Int)[]) : Int[][] {
    // ...
    return [];
}

### Task 4.2. Extract a list of triangles from an adjacency matrix


**Inputs:**

   1. The number of vertices in the graph $V (V \leq 6)$.
   
   2. An adjacency matrix describing the graph in the format from task 4.1.
   
**Output:** 

An array of 3-tuples listing all triangles in the graph, that is, all triplets of vertices connected by edges. Each of the 3-tuples should list the triangle vertices in ascending order, and the 3-tuples in the array should be sorted in ascending order as well.

**Example:** 

Consider the adjacency matrix
$$
\begin{bmatrix}
-1 &  0 &  1 \\
 0 & -1 &  2 \\
 1 &  2 & -1
\end{bmatrix}
$$

The list of triangles for it would be `[(0, 1, 2)]`.

In [ ]:
%kata T42_AdjacencyMatrixAsTrianglesList

function AdjacencyMatrixAsTrianglesList (V : Int, adjacencyMatrix : Int[][]) : (Int, Int, Int)[] {
    // ...
    return [];
}

### Task 4.3. Classical verification of triangle-free coloring
**Inputs:**
   1. The number of vertices in the graph $V (V \leq 6)$.
   
   2. An array of $E$ tuples of integers, representing the edges of the graph $(E \leq 12)$. Each tuple gives the indices of the start and the end vertices of the edge. The vertices are indexed $0$ through $V - 1$.
   
   3. An array of $E$ integers, representing the edge coloring of the graph. $i^{th}$ element of the array is the color of the edge number $i$, and it is $0$ or $1$. The colors of edges in this array are given in the same order as the edges in the "edges" array.

**Output:** 

`true` if the given coloring is triangle-free (i.e., no triangle of edges connecting 3 vertices has all three edges in the same color), and `false` otherwise.

**Example:** 

Consider a graph with $V = 3$ and edges = `[(0, 1), (0, 2), (1, 2)]`. Some of the valid colorings for it would be `[0, 1, 0]` and `[-1, 5, 18]`.

In [ ]:
%kata T43_IsVertexColoringTriangleFree

function IsVertexColoringTriangleFree (V : Int, edges: (Int, Int)[], colors: Int[]) : Bool {
    // ...
    return true;
}

### Task 4.4. Oracle to check that three colors don't form a triangle (f(x) = 1 if at least two of three input bits are different)

**Inputs:**

  1. a 3-qubit array `inputs`,
  
  2. a qubit `output`.
  
**Goal:** 

Flip the output qubit if and only if at least two of the input qubits are different.

**Example:** 

For the result of applying the operation to state $(|001\rangle + |110\rangle + |111\rangle) \otimes |0\rangle$ will be $|001\rangle \otimes |1\rangle + |110\rangle \otimes |1\rangle + |111\rangle \otimes |0\rangle$.

In [ ]:
%kata T44_ValidTriangleOracle

operation ValidTriangleOracle (inputs : Qubit[], output : Qubit) : Unit is Adj+Ctl {
    // ...
}

### Task 4.5. Oracle for verifying triangle-free edge coloring (f(x) = 1 if the graph edge coloring is triangle-free)

**Inputs:**
  1. The number of vertices in the graph $V (V \leq 6)$.
  
  2. An array of $E$ tuples of integers `edges`, representing the edges of the graph ($0 \leq E \leq V(V-1)/2$). Each tuple gives the indices of the start and the end vertices of the edge. The vertices are indexed $0$ through $V - 1$. The graph is undirected, so the order of the start and the end vertices in the edge doesn't matter.
  
  3. An array of $E$ qubits `colorsRegister` that encodes the color assignments of the edges. Each color will be $0$ or $1$ (stored in 1 qubit). The colors of edges in this array are given in the same order as the edges in the `edges` array.
  
  4. A qubit `target` in an arbitrary state.

**Goal:** 

Implement a marking oracle for function $f(x) = 1$ if the coloring of the edges of the given graph described by this colors assignment is triangle-free, i.e., no triangle of edges connecting 3 vertices has all three edges in the same color.

In this task you are not allowed to use quantum gates that use more qubits than the number of edges in the graph,
unless there are 3 or less edges in the graph. For example, if the graph has 4 edges, you can only use 4-qubit gates or less.
You are guaranteed that in tests that have 4 or more edges in the graph the number of triangles in the graph 
will be strictly less than the number of edges.

**Example:** 
A graph with 3 vertices and 3 edges `[(0, 1), (1, 2), (2, 0)]` has one triangle.  
The result of applying the operation to state $\frac{1}{\sqrt{3}} \big(|001\rangle + |110\rangle + |111\rangle\big) \otimes |0\rangle$ will be $\frac{1}{\sqrt{3}} \big(|001\rangle \otimes |1\rangle + |110\rangle \otimes |1\rangle + |111\rangle \otimes |0\rangle\big)$.
The first two terms describe triangle-free colorings, and the last term describes a coloring where all edges of the triangle have the same color.

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  Make use of functions and operations you've defined in previous tasks. 
</details>


In [ ]:
%kata T45_TriangleFreeColoringOracle

operation TriangleFreeColoringOracle (
    V : Int, 
    edges : (Int, Int)[], 
    colorsRegister : Qubit[], 
    target : Qubit
) : Unit is Adj+Ctl {
    // ...
}